In [4]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from nltk.corpus import gutenberg, stopwords
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
%matplotlib inline

---

Choosing which authors to use

In [5]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [26]:
#Choosing 10 authors
austen = gutenberg.raw('austen-persuasion.txt')
bryant = gutenberg.raw('bryant-stories.txt')
blake = gutenberg.raw('blake-poems.txt')
burgs = gutenberg.raw('burgess-busterbrown.txt')
chests = gutenberg.raw('chesterton-ball.txt')
carroll = gutenberg.raw('carroll-alice.txt')
edges = gutenberg.raw('edgeworth-parents.txt')
shakes = gutenberg.raw('shakespeare-macbeth.txt')
milton = gutenberg.raw('milton-paradise.txt')
whitman = gutenberg.raw('whitman-leaves.txt')

---

Data Cleaning

In [27]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    text = re.sub(r'CHAPTER \d+', '', text)
    text = re.sub("\\n\\n.*?\\n\\n", '', text)
  
    # Get rid of extra whitespace.
    text = ' '.join(text.split())

    
    return text

In [28]:
# Clean all documents
austen = text_cleaner(austen)
bryant = text_cleaner(bryant)
blake = text_cleaner(blake)
burgs = text_cleaner(burgs)
chests = text_cleaner(chests)
carroll = text_cleaner(carroll)
edges = text_cleaner(edges)
shakes = text_cleaner(shakes)
milton = text_cleaner(milton)
whitman = text_cleaner(whitman)

In [31]:
# run spacy and analyze the documents
nlp = spacy.load('en')

austen_doc = nlp(austen)
bryant_doc = nlp(bryant)
blake_doc = nlp(blake)
burgs_doc = nlp(burgs)
chests_doc = nlp(chests)
carroll_doc = nlp(carroll)
edges_doc = nlp(edges)
shakes_doc = nlp(shakes)
milton_doc = nlp(milton)
whitman_doc = nlp(whitman)

In [34]:
# Group into sentences.
austen_sents = [[sent, "Austen"] for sent in austen_doc.sents]
bryant_sents = [[sent, "Bryant"] for sent in bryant_doc.sents]
blake_sents = [[sent, "Blake"] for sent in blake_doc.sents]
burgs_sents = [[sent, "Burgs"] for sent in burgs_doc.sents]
chests_sents = [[sent, "Chests"] for sent in chests_doc.sents]
carroll_sents = [[sent, "Carroll"] for sent in carroll_doc.sents]
edges_sents = [[sent, "Edges"] for sent in edges_doc.sents]
shakes_sents = [[sent, "Shakes"] for sent in shakes_doc.sents]
milton_sents = [[sent, "Milton"] for sent in milton_doc.sents]
whitman_sents = [[sent, "Whitman"] for sent in whitman_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(austen_sents + bryant_sents + blake_sents + burgs_sents + 
                         chests_sents + carroll_sents + edges_sents + shakes_sents + milton_sents +
                        whitman_sents)
sentences.head()

,0,1
0,"(Sir, Walter, Elliot, ,, of, Kellynch, Hall, ,...",Austen
1,"(This, was, the, page, at, which, the, favouri...",Austen
2,"("")",Austen
3,"(Precisely, such, had, the, paragraph, origina...",Austen
4,"(Then, followed, the, history, and, rise, of, ...",Austen


In [35]:
len(sentences)

40779

---

Bag of Words

In [36]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
austenwords = bag_of_words(austen_doc)
bryantwords = bag_of_words(bryant_doc)
blakewords = bag_of_words(blake_doc)
burgswords = bag_of_words(burgs_doc)
chestswords = bag_of_words(chests_doc)
carrollwords = bag_of_words(carroll_doc)
edgeswords = bag_of_words(edges_doc)
shakeswords = bag_of_words(shakes_doc)
miltonwords = bag_of_words(milton_doc)
whitmanwords = bag_of_words(whitman_doc)

# Combine bags to create a set of unique words.
common_words = set(austenwords + bryantwords + blakewords + burgswords + 
                   chestswords + carrollwords + edgeswords + shakeswords + miltonwords + whitmanwords)

# How many words we got?
len(common_words)

8414

In [ ]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
